## 准备数据

In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets, Model

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    # Add a channels dimension
    x = x[..., tf.newaxis].astype("float32")
    x_test = x_test[..., tf.newaxis].astype("float32")
    # 使用 tf.data 来将数据集切分为 batch 以及混淆数据集：
    train_ds = tf.data.Dataset.from_tensor_slices(
    (x, y)).shuffle(10000).batch(32)
    test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1000)
    return train_ds, test_ds

## 建立模型

In [6]:
class myModel(Model):
    def __init__(self):
        super(myModel, self).__init__()
        #################### 填空一
        '''声明模型对应的参数'''
        #卷积层1： patch 7x7, in size 1, out size 32，  滑动步长 1， padding  same, 激活函数 relu (tf.keras.layers.Conv2D)
        self.conv1 = layers.Conv2D(filters=32, kernel_size=7, strides=1, padding='same', activation='relu')
        #池化层1：  滑动步长 是 2步; 池化窗口的尺度 高和宽度都是2; padding 方式  same (layers.MaxPool2D)
        self.pool1 = layers.MaxPool2D(pool_size=2, strides=2, padding='same')
        #卷积层2： patch 5x5, in size 32, out size 64，  滑动步长 1， padding 方式 same, 激活函数 relu
        self.conv2 = layers.Conv2D(filters=64, kernel_size=5, strides=1, padding='same', activation='relu')
        #池化层2：  滑动步长 是 2步; 池化窗口的尺度 高和宽度都是2; padding 方式  same
        self.pool2 = layers.MaxPool2D(pool_size=2, strides=2, padding='same')
        # 铺平特征映射
        self.flatten = layers.Flatten()
        # 全连接层 1: output dim 1024, 激活函数 relu： (tf.keras.layers.Dense)
        self.d1 = layers.Dense(1024, activation='relu')
        # 全连接层 2: output dim 10：
        self.d2 = layers.Dense(10)  # 输出未归一化的 logits

        ####################
    def call(self, x, training=False):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        return x
        ####################
        return logits
        
model = myModel()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

## 计算 loss

In [ ]:
# 创建训练损失的平均值记录器（自动累加并计算均值）
train_loss = tf.keras.metrics.Mean(name='train_loss')

# 创建训练准确率记录器（适用于标签为整数类别，如 0,1,2... 而非 one-hot）
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

# 创建测试损失的平均值记录器
test_loss = tf.keras.metrics.Mean(name='test_loss')

# 创建测试准确率记录器（同样适用于整数标签）
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


# @tf.function 装饰器：将 Python 函数编译为高效的 TensorFlow 图（提升运行速度）
@tf.function
def train_one_step(model, optimizer, x, y):
    # 使用 GradientTape 自动记录前向计算过程，以便后续计算梯度
    with tf.GradientTape() as tape:
        # 前向传播：training=True 表示启用训练行为（如 Dropout、BatchNorm 更新）
        predictions = model(x, training=True)
        # 计算损失（loss_object 应为已定义的损失函数，如 SparseCategoricalCrossentropy）
        loss = loss_object(y, predictions)

    # 自动计算所有可训练参数关于 loss 的梯度
    gradients = tape.gradient(loss, model.trainable_variables)
    
    # 使用优化器将梯度应用到模型参数上（即：参数更新）
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # 更新训练损失和准确率的累积统计值（内部自动累加并计数）
    train_loss(loss)          # 将当前 batch 的 loss 加入平均计算
    train_accuracy(y, predictions)  # 将当前 batch 的预测与真实标签比较，更新准确率


# @tf.function：同样编译为图模式，加速测试过程
@tf.function
def test_step(model, images, labels):
    # 前向传播：training=False 表示禁用训练行为（如 Dropout 关闭，BatchNorm 用历史统计量）
    predictions = model(images, training=False)
    # 计算当前 batch 的损失
    t_loss = loss_object(labels, predictions)

    # 更新测试损失和准确率的累积统计值
    test_loss(t_loss)               # 累积测试损失
    test_accuracy(labels, predictions)  # 累积测试准确率

## 实际训练

In [ ]:
# 加载 MNIST 数据集：train_ds 是训练数据集（带标签），test_ds 是测试数据集
# 通常返回的是 tf.data.Dataset 对象，已预处理（归一化、batch 等）
train_ds, test_ds = mnist_dataset()

# 设置训练总轮数（epochs）
EPOCHS = 5

# 开始训练循环：每一轮（epoch）遍历整个训练集一次
for epoch in range(EPOCHS):
    #在每个 epoch 开始前，重置所有指标的累积状态
    # 否则 loss 和 accuracy 会持续累加上一轮的结果，导致数值错误
    train_loss.reset_state()        # 清空训练损失的累计值
    train_accuracy.reset_state()    # 清空训练准确率的累计值
    test_loss.reset_state()         # 清空测试损失的累计值
    test_accuracy.reset_state()     # 清空测试准确率的累计值

    #遍历训练数据集中的每一个 batch
    # images: 形状如 [batch_size, 28, 28, 1] 的图像张量
    # labels: 形状如 [batch_size] 的整数标签（0~9）
    for images, labels in train_ds:
        # 执行一个训练步骤：
        # - 前向传播 + 计算损失
        # - 自动求导 + 更新参数
        # - 累积当前 batch 的 loss 和 accuracy 到指标中
        train_one_step(model, optimizer, images, labels)

    #遍历整个测试数据集（用于评估当前模型性能）
    # 注意：测试时通常不计算梯度，也不更新参数
    for test_images, test_labels in test_ds:
        # 执行一个测试步骤：
        # - 前向传播（training=False）
        # - 计算损失和准确率
        # - 累积到测试指标中
        test_step(model, test_images, test_labels)

    #打印当前 epoch 的训练和测试结果
    # .result() 返回当前 epoch 所有 batch 的平均 loss / 平均 accuracy
    print(
        f'Epoch {epoch + 1}, '                          # 当前是第几轮（从1开始）
        f'Loss: {train_loss.result():.4f}, '            # 训练平均损失（保留4位小数）
        f'Accuracy: {train_accuracy.result() * 100:.2f}%, '  # 训练准确率（转为百分比）
        f'Test Loss: {test_loss.result():.4f}, '        # 测试平均损失
        f'Test Accuracy: {test_accuracy.result() * 100:.2f}%' # 测试准确率（百分比）
    )

Epoch 1, Loss: 0.10241908580064774, Accuracy: 96.80332946777344, Test Loss: 0.05334934592247009, Test Accuracy: 98.15999603271484
Epoch 2, Loss: 0.03817015141248703, Accuracy: 98.83000183105469, Test Loss: 0.03605320304632187, Test Accuracy: 98.83999633789062
Epoch 3, Loss: 0.025063052773475647, Accuracy: 99.16999816894531, Test Loss: 0.0350714735686779, Test Accuracy: 98.90999603271484
Epoch 4, Loss: 0.019615016877651215, Accuracy: 99.38166809082031, Test Loss: 0.037959299981594086, Test Accuracy: 99.0
Epoch 5, Loss: 0.016512813046574593, Accuracy: 99.50666809082031, Test Loss: 0.04503753036260605, Test Accuracy: 98.68000030517578
